# Model With GPU (Fine tuning the model using LoRA)

In [33]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, GenerationConfig
import torch
from peft import LoraConfig, TaskType, get_peft_model, PeftModel, PeftConfig
from tqdm import tqdm
import evaluate

## Import the data from Hugging face

In [2]:
# dataset = load_dataset('samhog/psychology-10k', split = ['train'])[0]
# dataset.save_to_disk('./dataset')

dataset = load_from_disk('./dataset')
dataset

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 9846
})

let's walk trough the data

In [3]:
def print_conv(me, model):
    for i, j in zip(me, model):
        print(
            f"""
Me : {i}

Param mitr : {j}\n\n
            """
        )

In [4]:
print_conv(dataset['input'][:3], dataset['output'][:3])


Me : I'm feeling really anxious lately and I don't know why.

Param mitr : It's common to feel anxious at times, and there can be many reasons for it. Have there been any recent changes or stressors in your life that may be contributing to your anxiety? Let's work together to identify any triggers and develop coping strategies to manage your anxiety.


            

Me : I think my partner may be cheating on me. What should I do?

Param mitr : It's understandable to feel worried and suspicious in this situation. Have you talked to your partner about your concerns? It's important to communicate openly and honestly with them. If you're still feeling uncertain, we can work on developing a plan to address the situation in a healthy and constructive way.


            

Me : I'm feeling really overwhelmed with work and school. I don't know how to manage my time and it's causing me a lot of stress.

Param mitr : It sounds like you're going through a difficult time. Let's work on creating a 

In [5]:
dataset.shape

(9846, 3)

In [6]:
Dataset = dataset.train_test_split(test_size = 0.2)
Dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 7876
    })
    test: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 1970
    })
})

## Import our model with its tokenizer

In [7]:
# if you want to use QLoRA use this but remember in 
# normal inference you have to use GPU

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_storage=torch.bfloat16,
# )

In [8]:
# Importing model from the Hugging Face hub

model_name = 'google/flan-t5-small'
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype = torch.bfloat16,
    # quantization_config = bnb_config # Use it if you want Quantization
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Saving the model

# model.save_pretrained('/kaggle/working/Model')
# tokenizer.save_pretrained('/kaggle/working/Model')

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Let's try some stuff with tokenizer

In [9]:
tokenizer("hey", return_token_type_ids = False)

{'input_ids': [3, 13133, 1], 'attention_mask': [1, 1, 1]}

In [10]:
device = torch.device('cuda')
tokenizer.decode(
    model.generate(
        tokenizer(
            dataset['input'][0], return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        )['input_ids'].to(device)
    )[0], skip_special_tokens= True
)

"I'm feeling really anxious lately and I don't know why."

In [11]:
def inference(input_data, param_mitr = model):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent + task for sent in input_data]
    output = param_mitr.generate(
        tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        )['input_ids'].to(device),
        generation_config = GenerationConfig(max_new_token = 200)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)

In [12]:
inference(dataset['input'][:4])


Me : I'm feeling really anxious lately and I don't know why.

Param mitr : I'm feeling very anxious.


            

Me : I think my partner may be cheating on me. What should I do?

Param mitr : I should give this patient a call.


            

Me : I'm feeling really overwhelmed with work and school. I don't know how to manage my time and it's causing me a lot of stress.

Param mitr : I'm feeling really overwhelmed with work and school. I don't know how to manage my


            

Me : I'm having trouble sleeping and I'm constantly tired. I think it might be because of my medication.

Param mitr : I'm having trouble sleeping and I'm constantly tired.


            


The model is showing good result on ICL (In Context Learning) with zero shot inference.

In [13]:
def tokenize_function(example):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + question + '\n' + task for question in example['input']]
    example['input_ids'] = tokenizer(inp, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example['output'], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example


tokenized_datasets = Dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['input', 'output', 'instruction'])

Map:   0%|          | 0/7876 [00:00<?, ? examples/s]

Map:   0%|          | 0/1970 [00:00<?, ? examples/s]

## Fine tune our model (Using LoRA)

In [14]:
config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r = 32,
    target_modules = "all-linear",
    lora_alpha=32,
    lora_dropout=0.05
)

In [15]:
Model = get_peft_model(model, config)
Model.print_trainable_parameters()

trainable params: 5,111,808 || all params: 82,072,960 || trainable%: 6.2284


In [16]:
TrainArgs = TrainingArguments(
    output_dir = './Model',
    learning_rate = 1e-3,
    num_train_epochs = 2,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    logging_steps=1,
    load_best_model_at_end = True,
    report_to = ['tensorboard']
)

In [17]:
trainer = Trainer(
    model = Model,
    args = TrainArgs,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test']
)

In [18]:
import gc
# To clear the GPU
def clean():
    for i in range(15):
      gc.collect()
      torch.cuda.empty_cache()

In [19]:
clean()
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.679700,1.628696
2,1.664100,1.618504


TrainOutput(global_step=3938, training_loss=1.8223896171914677, metrics={'train_runtime': 1268.6262, 'train_samples_per_second': 12.417, 'train_steps_per_second': 3.104, 'total_flos': 3175508355317760.0, 'train_loss': 1.8223896171914677, 'epoch': 2.0})

In [20]:
def peft_infrence_with_GPU(input_data, model = Model):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent +'\n'+ task for sent in input_data]
    output = model.generate(
        **tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        ).to('cuda'),
        generation_config = GenerationConfig(max_new_tokens=100, num_beams=1)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)

In [22]:
peft_infrence_with_GPU(Dataset['test']['input'][:4])


Me : I feel like I'm not good enough compared to my colleagues at work. What can I do to boost my confidence?

Param mitr : It's important to identify your strengths and weaknesses and develop a plan to improve your confidence. Consider seeking professional help from a therapist or a therapist to work through any underlying issues that may be contributing to your lack of confidence. Additionally, practicing self-care and identifying negative self-talk can also help boost confidence.


            

Me : I've been feeling really down lately and don't know why. Can you help me figure it out?

Param mitr : It's important to identify the cause of your depression and develop coping strategies to manage it. Let's work on developing coping strategies and identifying any underlying causes that may be contributing to your depression.


            

Me : I've been feeling really down and hopeless lately. What can I do to feel better?

Param mitr : It's important to seek professional help to id

## Evaluation

In [55]:
model_old = AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/Model',
                                              device_map="auto",
                                              torch_dtype = torch.bfloat16
                                             )

In [50]:
def inference(test_data, model_):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent +'\n'+ task for sent in test_data]
    output = model_.generate(
        **tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        ).to('cuda'),
        generation_config = GenerationConfig(max_new_tokens=200, num_beams=1)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    return decoded

In [56]:
rouge = evaluate.load('rouge')
predicted_new = inference(Dataset['test']['input'][:10], model_ = Model)
predicted_old = inference(Dataset['test']['input'][:10], model_ = model_old)
human_base_line = Dataset['test']['output'][:10]

In [57]:
rouge_score_old = rouge.compute(
    predictions = predicted_old,
    references = human_base_line
)
rouge_score_old

{'rouge1': 0.11607087438609179,
 'rouge2': 0.031482078086114904,
 'rougeL': 0.0879792724720261,
 'rougeLsum': 0.08786995168349267}

In [58]:
rouge_score_new = rouge.compute(
    predictions = predicted_new,
    references = human_base_line
)
rouge_score_new

{'rouge1': 0.42365145391096454,
 'rouge2': 0.21080558265702876,
 'rougeL': 0.32149324267318413,
 'rougeLsum': 0.3200334978676107}

## Save the PEFT Adapter

In [23]:
trainer.model.save_pretrained('./peft_for_param_mitr')

# Model without GPU (using saved fine-tuned model)

## Load model from local storage

In [24]:
tokenizer = AutoTokenizer.from_pretrained('/kaggle/working/Model', device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/Model',
                                              device_map="auto",
                                              torch_dtype = torch.bfloat16
                                             )

## Load PEFT adapter to the base model

In [25]:
peft_model_path = './peft_for_param_mitr'
Model = PeftModel.from_pretrained(
            model,
            peft_model_path,
            is_trainable = False
            )

**These Cell will only work when you do not have GPU**

In [26]:
def peft_infrence(input_data, model = Model):
    intruct = dataset['instruction'][0]
    task = "Answer :"
    inp = [intruct + "\n" + sent +'\n'+ task for sent in input_data]
    output = model.generate(
        **tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        ),
        generation_config = GenerationConfig(max_new_tokens=200, num_beams=1)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)

In [28]:
# peft_infrence(Dataset['test']['input'][:4])